<a href="https://colab.research.google.com/github/SophiaU/Data-Sanitization/blob/main/Vehicle_Loan_Repaid_Amounts_SA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
# install and import pygsheets
!pip install -q pygsheets
!pip install --upgrade -q pip
!pip install -q wheel
!pip install -q redshift_connector
!pip install -q sqlalchemy
!pip install -q psycopg2
!apt-get install -y sshpass

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 150.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [redshift_connector]
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  sshpass
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 11.7 kB of archives.
After this operation, 35.8 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 sshpass amd64 1.09-1 [11.7 kB]
Fetched 11.7 kB in 0s (263 kB/s)
Selecting previously unselected package sshpass.
(Reading database ... 126281 files and directories currently installed.)
Preparing to unpack .../sshpass_1.09-1_amd64.deb ...
Unpacking sshpass (1.09-1) ...
Setting up sshpass (1.09-1) ...
Processing tr

In [ ]:
import psycopg2
import pygsheets
import gspread
import pandas as pd
import numpy as np
from functools import reduce
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import credentials
from google.oauth2 import _client
from pydrive.auth import GoogleAuth
from oauth2client.service_account import ServiceAccountCredentials
from google.auth import default
import glob
import json
import datetime
from google.oauth2 import service_account

In [ ]:
# Authentication
creds, _ = default()

gc = gspread.authorize(creds)

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

jsonfile = drive.CreateFile({'id':'1sttqJM-JL9yC0DB1xoC_-__5ztM3iAVi'})

jsonfile.GetContentFile('service_account.json')

with open('service_account.json') as source:
   info = json.load(source)
credentials = service_account.Credentials.from_service_account_info(info)

client = pygsheets.authorize(service_account_file='service_account.json')

# Redshift Authentication
from google.colab import userdata

private_key = userdata.get('ssh_private_key')

private_key_path = '/root/.ssh/ssh.pem'
!mkdir -p ~/.ssh
with open(private_key_path, 'w') as f:
    f.write(private_key)
!chmod 600 /root/.ssh/ssh.pem


#!kill -9 $(sudo lsof -t -i:5439)
!ssh -i /root/.ssh/ssh.pem -p 45022 -o StrictHostKeyChecking=no -fNL 127.0.0.1:5439:dwh.moovedata.io:5439 google_collab@sshtunnel.moovedata.io

#!kill -9 $(sudo lsof -t -i:5439)
# !sshpass -p 'AEL4wZYjAuaNKwBqdfHJ$' ssh -p 45022 -o StrictHostKeyChecking=no -fNL 127.0.0.1:5439:dwh.moovedata.io:5439 google_collab@sshtunnel.moovedata.io

In [ ]:
# Access the DB Reconciliation Data
import pandas as pd
import datetime
import psycopg2

# def redshift(x):
#     conn = psycopg2.connect(
#             host = "127.0.0.1",
#             port = "5439",
#             database = "dev",
#             user = "sophia_uwoziya", #add your user
#             password = "QtIuPhaxN5pwvRdC") # add your pw
#     # Execute the query
#     query = x
#     x = pd.read_sql(query, conn)
#     conn.close()
#     return x

#function to query db
def redshift(query_sa):
  redshift_password = userdata.get('redshift_password')
  conn = psycopg2.connect(
      host = '127.0.0.1',
      port = 5439,
      database = 'dev',
      user = 'read-all',
      password = redshift_password
  )
  data = pd.read_sql(query_sa, conn)
  conn.close()
  return data

In [ ]:
from datetime import datetime, timedelta

# Function to find the Monday of the week for any given date
def get_monday(date):
    if pd.notna(date):
        return date - timedelta(days=date.weekday())
    return np.nan  # Return NaN if the date is NaN


def get_sunday(date):
    # Calculating how many days to add to the date to reach Sunday (weekday() returns 0 for Monday, 6 for Sunday)
    days_until_sunday = 6 - date.weekday()  # Sunday is 6
    return date + timedelta(days=days_until_sunday)

In [ ]:
# Old Query no longer run
# query = """
# SELECT
# week, drn, product, CAST(total_remitted_for_the_loan_emi_opex_ AS double precision) AS total_loan_repaid
# FROM fivetran_gsheets.sa_historical_loanwaterfall
# WHERE drn is not null
# UNION ALL
# SELECT
# week, drn,product,  CAST(total_remitted_for_the_loan_emi_opex_ AS double precision) AS total_loan_repaid
# FROM fivetran_gsheets.sa_current_loanwaterfall
# where week > '2023-09-25' --and '2024-07-29' and drn is not null

# """

# loan_waterfall = redshift(query)

In [ ]:
query = """
SELECT
  CAST(week AS DATE) AS week,
  drn,
  CAST(total_remitted_for_the_loan_emi_opex_ AS double precision) AS total_loan_repaid
FROM fivetran_gsheets.sa_historical_loanwaterfall
UNION all
SELECT
  CAST(week AS DATE) AS week,
  drn,
  CAST(total_remitted_for_the_loan_emi_opex_ AS double precision) AS total_loan_repaid
FROM fivetran_gsheets.sa_historical_loanwaterfall_2 shl2
  WHERE CAST(week AS DATE) >= '2023-10-02'
UNION all
  SELECT
  CAST(week AS DATE) AS week,
  drn,
  CAST(total_remitted_for_the_loan_emi_opex_ AS double precision) AS total_loan_repaid
FROM fivetran_gsheets.sa_historical_loanwaterfall_3 shl3
  where week NOT LIKE '%/%' and CAST(week AS DATE) >= '2024-07-01'
UNION all
  SELECT
  CAST(week AS DATE) AS week,
  drn,
  CAST(total_remitted_for_the_loan_emi_opex_ AS double precision) AS total_loan_repaid
FROM fivetran_gsheets.sa_current_loanwaterfall
WHERE week NOT LIKE '%/%'
  AND CAST(week AS DATE) >= '2025-03-03'
ORDER BY week
"""

loan_waterfall = redshift(query)

/tmp/ipython-input-7-1611149217.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query_sa, conn)


In [ ]:
loan_waterfall['week'] = pd.to_datetime(loan_waterfall['week'], format='%Y-%m-%d')
loan_waterfall.sample(10)

,week,drn,total_loan_repaid
277362,2025-03-24,MVS20157,1226.86
125334,2024-02-26,MVS15299,2006.65
112069,2024-01-15,MVS17302,2800.00
39540,2022-12-26,MVS12607,2300.00
219449,2024-11-18,MVS18368,3100.00
86905,2023-10-16,MVS00102,2145.83
24528,2022-09-05,MVS10837,1844.75
68479,2023-06-26,MVS14212,2575.00
146908,2024-05-13,MVS15083,1450.00
18526,2022-07-11,MVS10287,1533.33


Running History Query for Specific Driver

In [ ]:
driver_drn = 'MVS12818'

driver_query = (f"""
WITH unified_loanwaterfall AS (
SELECT
  CAST(week AS DATE) AS week,
  drn, name, product, city, vehicle_model,
  effective_days,
  CAST(uber_balance AS DOUBLE PRECISION),
  CAST(payins AS DOUBLE PRECISION),
  CAST(payouts AS DOUBLE PRECISION),
  CAST(extra_charges AS DOUBLE PRECISION),
  CAST(total_week_outstanding AS DOUBLE PRECISION),
  CAST(expected_remittance AS DOUBLE PRECISION),
  CAST(total_received AS DOUBLE PRECISION),
  CAST(total_remitted_for_the_loan_emi_opex_ AS DOUBLE PRECISION)
FROM fivetran_gsheets.sa_historical_loanwaterfall
union all
SELECT
  CAST(week AS DATE) AS week,
  drn, name, product, city, vehicle_model,
  effective_days,
  CAST(uber_balance AS DOUBLE PRECISION),
  CAST(payins AS DOUBLE PRECISION),
  CAST(payouts AS DOUBLE PRECISION),
  CAST(extra_charges AS DOUBLE PRECISION),
  CAST(total_week_outstanding AS DOUBLE PRECISION),
  CAST(expected_remittance AS DOUBLE PRECISION),
  CAST(total_received AS DOUBLE PRECISION),
  CAST(total_remitted_for_the_loan_emi_opex_ AS DOUBLE PRECISION)
FROM fivetran_gsheets.sa_historical_loanwaterfall_2 shl2
  WHERE CAST(week AS DATE) >= '2023-10-02'
union all
SELECT
  CAST(week AS DATE) AS week,
  drn, name, product, city, vehicle_model,
  effective_days,
  CAST(uber_balance AS DOUBLE PRECISION),
  CAST(payins AS DOUBLE PRECISION),
  CAST(payouts AS DOUBLE PRECISION),
  CAST(extra_charges AS DOUBLE PRECISION),
  CAST(total_week_outstanding AS DOUBLE PRECISION),
  CAST(expected_remittance AS DOUBLE PRECISION),
  CAST(total_received AS DOUBLE PRECISION),
  CAST(total_remitted_for_the_loan_emi_opex_ AS DOUBLE PRECISION)
FROM fivetran_gsheets.sa_historical_loanwaterfall_3 shl3
  where week NOT LIKE '%/%' and CAST(week AS DATE) >= '2024-07-01'
 union ALL
SELECT
  CAST(week AS DATE) AS week,
  drn, name, product, city, vehicle_model,
  effective_days,
  CAST(uber_balance AS DOUBLE PRECISION),
  CAST(payins AS DOUBLE PRECISION),
  CAST(payouts AS DOUBLE PRECISION),
  CAST(extra_charges AS DOUBLE PRECISION),
  CAST(total_week_outstanding AS DOUBLE PRECISION),
  CAST(expected_remittance AS DOUBLE PRECISION),
  CAST(total_received AS DOUBLE PRECISION),
  CAST(total_remitted_for_the_loan_emi_opex_ AS DOUBLE PRECISION)
FROM fivetran_gsheets.sa_current_loanwaterfall
WHERE week >= '2025-03-03' AND week NOT LIKE '%/%')
-- Final filter and sort
SELECT *
FROM unified_loanwaterfall
WHERE drn = '{driver_drn}'   --week >= '2024-01-01' AND
ORDER BY week;
""")



driver_history = redshift(driver_query)
driver_history.head(10)
totals = driver_history[driver_history['drn'] == driver_drn]['total_remitted_for_the_loan_emi_opex_'].sum()
# totals = driver_history[driver_history['drn'].isin(['MVS11092', 'MVS12818'])].groupby('drn')['total_remitted_for_the_loan_emi_opex_'].sum()

print(totals)
#

<ipython-input-13-2a8768b0a61a>:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query_sa, conn)


In [ ]:
query1 = """
WITH assignments AS (
  select registration_no, drn,
  	vehicle_assignment_start_date as effective_start_date,
  	vehicle_assignment_end_date as effective_end_date,
  	vehicle_updated_by
  		from dbt.core__vehicle_driver_assignment
) ,
vehicleAssign as (
    select row_number() over (partition by drn order by effective_start_date desc),*
    from assignments)
SELECT
  va.registration_no AS "plate_number",
  va.drn,
  COALESCE(dd.first_name,'') || ' ' || COALESCE(dd.middle_name,'') || ' ' || COALESCE(dd.last_name,'') AS "driver_name",
  effective_start_date,
  effective_end_date,
  v.make, v.model_name, v.model_launch_year as year, v.color,
  v.city,
  v.vin AS vin_no,
  v.country,
  v.carco_name AS car_co,
  vp.partner_product_name AS vehicle_plan,
  v.vehicle_status,
  vehicle_updated_by AS "Who Assigned the vehicle"
FROM
  vehicleAssign va
LEFT JOIN dbt.core__driver_domain dd ON dd.drn = va.drn
LEFT JOIN dbt.core__vehicle_domain v ON va.registration_no = v.registration_no
LEFT JOIN dbt.core__vehicle_product_plan_assignment vp ON va.registration_no = vp.registration_no
where lower(v.country) in ('south africa')
and effective_start_date is not null
order by effective_end_date desc, effective_start_date desc
"""
vehicle_assignment = redshift(query1)
vehicle_assignment.sample(10)

/tmp/ipython-input-7-1611149217.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query_sa, conn)


,plate_number,drn,driver_name,effective_start_date,effective_end_date,make,model_name,year,color,city,vin_no,country,car_co,vehicle_plan,vehicle_status,who assigned the vehicle
1897,KY01GLGP,MVS15421,MBEWE JAIRUS,2024-11-23 08:00:00.000,NaT,Toyota,Agya,2022,White,Johannesburg,MHKA4DA5N0J006569,South Africa,MVASA CCO 3,Uber Go,ACTIVE,stephanie.ferris@moove.io
5831,KV26CWGP,MVS19347,BOITUMELO JACOB MASILO,2024-10-07 09:45:00.000,2025-02-17 06:15:00,Toyota,Agya,2022,White,Johannesburg,MHKA4DA5N0J005992,South Africa,MVASA CCO 3,Uber Go,ACTIVE,kenneth.mahantji@moove.io
13052,KL56VLGP,MVS15641,RONNY MOKGALOBONE,2023-02-13 07:00:00.000,2023-08-03 09:00:00,Toyota,Agya,2022,White,Johannesburg,MHKA4DA5N0J004353,South Africa,MVASA CCO 2,Uber Go,ACTIVE,None
9255,LV03SRGP,MVS18681,NHLANHLA DERRICK ABEL NGUBANE,2024-08-08 08:00:00.000,2024-08-15 05:00:00,Hyundai,Grand i10,2023,White,Johannesburg,MALB251ALRM486611,South Africa,MVASA CCO 3,Uber Go,WRITE_OFF,tebogo.mthethwa@moove.io
12576,KT88MWGP,MVS15343,SIBUSISO MASEKO,2023-09-06 06:00:00.000,2023-09-22 00:00:00,Toyota,Agya,2022,White,Johannesburg,MHKA4DA5N0J005710,South Africa,MVASA CCO 3,Uber Go,ACTIVE,None
6493,KL16GYGP,MVS12512,BENJAMIN TARUGARIRA,2022-03-04 12:00:00.000,2025-01-16 08:00:00,Toyota,Agya,2022,White,Johannesburg,MHKA4DA5N0J003572,South Africa,MVASA CCO 2,Uber Go,ACTIVE,thandi.poolo@moove.io
2815,LW29TGGP,MVS20294,Gwyneth Gwen Mashele,2025-06-17 09:00:00.000,2025-07-10 02:00:00,Hyundai,Grand i10,2023,White,Johannesburg,MALB251ALRM521137,South Africa,MVASA CCO 3,Uber Go,ACTIVE,faeez.bulbulia@moove.io
11961,KT64CTGP,MVS17236,LUNGISANI SIBANDA,2023-11-22 16:00:00.000,2023-11-29 00:00:00,Toyota,Agya,2022,Silver,Johannesburg,MHKA4DA5N0J005638,South Africa,MVASA CCO 3,Uber Go,WRITE_OFF,None
1290,KT29XRGP,MVS21229,Dyondzani Edmond Ngwana,2025-03-28 08:00:00.000,NaT,Toyota,Agya,2022,White,Johannesburg,MHKA4DA5N0J005452,South Africa,MVASA CCO 3,Uber Go,ACTIVE,stephanie.ferris@moove.io
2038,CAA402718,MVS15456,EDWINA EDITH DAVIDS,2024-10-17 08:00:43.051,NaT,Suzuki,S-presso,2022,Silver,Cape Town,MA3RFL41S00354871,South Africa,MVASA CCO 2,Uber Go,ACTIVE,ihtishaam.adams@moove.io


In [ ]:
driver_vehicle = vehicle_assignment[['effective_start_date','drn','driver_name','plate_number','city','effective_end_date','make','model_name']]
driver_vehicle.sample(10)

,effective_start_date,drn,driver_name,plate_number,city,effective_end_date,make,model_name
11566,2023-10-14 10:00:00.000,MVS16287,PABALLO WILFRED SELOLO,LK18NWGP,Johannesburg,2024-01-02 00:00:00,Hyundai,Grand i10
11464,2023-10-24 10:00:00.000,MVS16855,NHLANHLA MSIZA MCEBISO,KD40PRGP,Johannesburg,2024-01-11 00:00:00,Toyota,Agya
1865,2024-11-29 14:30:00.000,MVS17312,TEBOGO ADOLPH TSELAPEDI,LK64ZXGP,Johannesburg,NaT,Hyundai,Grand i10
12167,2023-10-27 00:00:01.000,MVS10383,SHAHIED SULEMAN,CAA346385,Cape Town,2023-11-10 00:00:00,Suzuki,S-presso
8075,2024-08-16 08:00:42.216,MVS18793,JAMIE LINKS,CAA403061,Cape Town,2024-10-23 08:00:00,Suzuki,S-presso
5870,2024-08-06 08:00:00.000,MVS11061,EUGENE NCUBE,KM80BNGP,Johannesburg,2025-02-14 08:00:00,Toyota,Agya
918,2025-05-19 13:27:00.000,MVS21601,Vumile Gladstone Xoseni,CAA402802,Cape Town,NaT,Suzuki,S-presso
11282,2023-10-12 11:00:00.000,MVS16746,SIYABONGA TRY PHAKATHI,KD31YSGP,Johannesburg,2024-01-26 00:14:00,Toyota,Agya
9741,2024-06-20 08:00:00.000,MVS18197,DOUGLAS NKWALI,LL13SSGP,Johannesburg,2024-06-25 22:00:00,Hyundai,Grand i10
2398,2024-06-05 08:00:00.000,MVS14113,WISTON NKOMO,KT30JNGP,Johannesburg,NaT,Toyota,Agya


In [ ]:
# Writing to loan repaid Sheet tab
# comp_sheet = client.open_by_key('1tkf9TSZ5DNddCrt3FM2VHJLvqOhlA-bBxx-J84lVkBk')
comp_sheet = client.open_by_key('18pdGDfh5zcRB8Ut4C3gnmTCTt-TkSCWqyA0AD6nfw-s')
driver_vehicle_data = comp_sheet.worksheet_by_title("driver<>vehicle")
driver_vehicle_data.set_dataframe(driver_vehicle,(1,1),copy_head = True,nan='')

In [ ]:
vehicle_assignment = vehicle_assignment[['drn','plate_number','effective_start_date','effective_end_date','city']]
vehicle_assignment['effective_start_date'] = pd.to_datetime(vehicle_assignment['effective_start_date']).dt.date
vehicle_assignment['effective_end_date'] = pd.to_datetime(vehicle_assignment['effective_end_date']).dt.date

/tmp/ipython-input-14-1536160971.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicle_assignment['effective_start_date'] = pd.to_datetime(vehicle_assignment['effective_start_date']).dt.date
/tmp/ipython-input-14-1536160971.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicle_assignment['effective_end_date'] = pd.to_datetime(vehicle_assignment['effective_end_date']).dt.date


In [ ]:
from datetime import date

# Fill any empty records with the current date
current_date = date.today()
vehicle_assignment['effective_start_date'].fillna(current_date, inplace=True)
vehicle_assignment['effective_end_date'].fillna(current_date, inplace=True)

/tmp/ipython-input-15-664805160.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  vehicle_assignment['effective_start_date'].fillna(current_date, inplace=True)
/tmp/ipython-input-15-664805160.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].met

In [ ]:
#sense check
vehicle_assignment[vehicle_assignment['drn']=='MVS10529']

,drn,plate_number,effective_start_date,effective_end_date,city
4720,MVS10529,CAA404276,2025-02-27,2025-04-09,Cape Town


In [ ]:
vehicle_assignment['start_week'] = vehicle_assignment['effective_start_date'].apply(lambda date: get_monday(date) if pd.notna(date) else None)
vehicle_assignment['end_week'] = vehicle_assignment['effective_end_date'].apply(lambda date: get_sunday(date) if pd.notna(date) else None)
vehicle_assignment.sample(10)

,drn,plate_number,effective_start_date,effective_end_date,city,start_week,end_week
8733,MVS10329,KV35FDGP,2024-01-23,2024-09-18,Johannesburg,2024-01-22,2024-09-22
9126,MVS17328,LK97ZCGP,2023-11-30,2024-08-26,Johannesburg,2023-11-27,2024-09-01
11295,MVS13893,CAA402711,2023-09-21,2024-01-26,Cape Town,2023-09-18,2024-01-28
14774,MVS12880,CAA311099,2022-05-04,2022-12-08,Cape Town,2022-05-02,2022-12-11
13686,MVS13870,KX76JCGP,2023-02-06,2023-05-22,Johannesburg,2023-02-06,2023-05-28
6652,MVS18930,CAA349027,2024-10-01,2025-01-08,Cape Town,2024-09-30,2025-01-12
16498,MVS11281,CAA346502,2021-12-21,2022-02-07,Cape Town,2021-12-20,2022-02-13
9116,MVS17769,KT98FVGP,2024-07-24,2024-08-26,Johannesburg,2024-07-22,2024-09-01
5234,MVS19115,CAA403696,2024-11-20,2025-03-13,Cape Town,2024-11-18,2025-03-16
1800,MVS19019,CAA311061,2024-12-19,2025-07-16,Cape Town,2024-12-16,2025-07-20


In [ ]:
from datetime import datetime, timedelta

# Calculate the start and end of the current week
today = datetime.today()
current_week_start = today - timedelta(days=today.weekday())  # Monday of the current week
current_week_end = current_week_start + timedelta(days=6)     # Sunday of the current week

# Create a list to hold the merged waterfall<>assignment records
merged_records = []

# Loop through each record in loan_waterfall
for idx, lw_row in loan_waterfall.iterrows():
    drn = lw_row['drn']
    # Convert 'week' to Timestamp to avoid comparison errors
    week = pd.Timestamp(lw_row['week'])

    # Find all matching records in vehicle assignment for the same DRN in loans
    matching_records = vehicle_assignment[vehicle_assignment['drn'] == drn]

    # Initialize variables for storing matched record
    matched_record = None

    # Loop through each matching record to check/assign the date range
    for _, sd_row in matching_records.iterrows():
        # Check if 'start_week' and 'end_week' are not None
        if pd.notna(sd_row['start_week']) and pd.notna(sd_row['end_week']):
            # Adjust Start Week and End Week by +/- 3 days
            adjusted_start_week = sd_row['start_week'] - timedelta(days=3)
            adjusted_end_week = sd_row['end_week'] + timedelta(days=3)

            # Ensure adjusted_start_week and adjusted_end_week are Timestamps for comparison
            adjusted_start_week = pd.Timestamp(adjusted_start_week)
            adjusted_end_week = pd.Timestamp(adjusted_end_week)
        else:
            # Set to current week's start and end dates if start_week or end_week is None
            adjusted_start_week = current_week_start
            adjusted_end_week = current_week_end

        # Check if the 'week' falls between the adjusted Start Week and End Week
        if adjusted_start_week and adjusted_end_week and adjusted_start_week <= week <= adjusted_end_week:
            matched_record = sd_row
            break  # We found a match, no need to check further

    # If a match is found, merge the loan waterfall record with vehicle assigned record
    if matched_record is not None:
        merged_row = {
            'drn': drn,
            'week': week,
            'total_loan_repaid': lw_row['total_loan_repaid'],
            'plate_number': matched_record['plate_number'],
            'effective_start_date': matched_record['effective_start_date'],
            'effective_end_date': matched_record['effective_end_date'],
            'start_week': matched_record['start_week'],
            'end_week': matched_record['end_week'],
            'Adjusted Start Week': adjusted_start_week,
            'Adjusted End Week': adjusted_end_week
        }
    else:
        # If no match is fopund for waterfall record, assign nulls for vehicle assigned columns
        merged_row = {
            'drn': drn,
            'week': week,
            'total_loan_repaid': lw_row['total_loan_repaid'],
            'plate_number': np.nan,
            'effective_start_date': np.nan,
            'effective_end_date': np.nan,
            'start_week': np.nan,
            'end_week': np.nan,
            'Adjusted Start Week': np.nan,
            'Adjusted End Week': np.nan
        }

    # Append the merged row to the list of merged records
    merged_records.append(merged_row)

# Convert the list of merged records to a DataFrame
merged_df = pd.DataFrame(merged_records)

# Display the final merged DataFrame
print("Final merged dataset with adjusted limits:")
merged_df.head(10)

Final merged dataset with adjusted limits:


,drn,week,total_loan_repaid,plate_number,effective_start_date,effective_end_date,start_week,end_week,Adjusted Start Week,Adjusted End Week
0,MVS10270,2021-09-13,612.0,KD40PTGP,2021-09-17,2022-04-01,2021-09-13,2022-04-03,2021-09-10,2022-04-06
1,MVS10169,2021-09-13,1855.0,KD09YVGP,2021-09-14,2023-03-31,2021-09-13,2023-04-02,2021-09-10,2023-04-05
2,MVS10189,2021-09-13,1835.0,KD09VPGP,2021-09-14,2022-07-25,2021-09-13,2022-07-31,2021-09-10,2022-08-03
3,MVS10224,2021-09-13,1841.0,KD04RMGP,2021-09-13,2022-05-18,2021-09-13,2022-05-22,2021-09-10,2022-05-25
4,MVS10041,2021-09-13,1824.0,KD09TSGP,2021-09-14,2022-04-21,2021-09-13,2022-04-24,2021-09-10,2022-04-27
5,MVS10200,2021-09-13,2246.0,KD09YSGP,2021-09-13,2024-06-09,2021-09-13,2024-06-09,2021-09-10,2024-06-12
6,MVS10221,2021-09-13,2217.0,KD04RKGP,2021-09-13,2022-03-20,2021-09-13,2022-03-20,2021-09-10,2022-03-23
7,MVS10272,2021-09-13,467.0,KD31WYGP,2021-09-17,2022-01-31,2021-09-13,2022-02-06,2021-09-10,2022-02-09
8,MVS10075,2021-09-13,1925.0,KD09VCGP,2021-09-14,2022-04-01,2021-09-13,2022-04-03,2021-09-10,2022-04-06
9,MVS10267,2021-09-13,519.0,KD31XTGP,2021-09-17,2024-06-03,2021-09-13,2024-06-09,2021-09-10,2024-06-12


In [ ]:
filtered_merged_df = merged_df[['plate_number','drn','week','total_loan_repaid']]
filtered_merged_df.sample(10)

,plate_number,drn,week,total_loan_repaid
308014,CAA403153,MVS10401,2025-06-02,2100.00
106430,KX70CPGP,MVS10199,2023-12-25,1237.50
84858,CAA151549,MVS16200,2023-10-09,2450.00
219886,LK55ZMGP,MVS17791,2024-11-18,1194.06
64305,KL34ZCGP,MVS15965,2023-06-05,2776.25
198477,LL23MDGP,MVS19331,2024-09-30,2583.33
4307,KD31WPGP,MVS10179,2022-01-17,2300.00
126872,LL02VWGP,MVS17228,2024-03-04,-179.31
206860,KD49PMGP,MVS10581,2024-10-21,2360.99
208827,CAA403188,MVS19223,2024-10-21,2450.00


In [ ]:
filtered_merged_df.shape

(321933, 4)

In [ ]:
# Writing to loan repaid Sheet tab
full_data = comp_sheet.worksheet_by_title("Data")
full_data.set_dataframe(filtered_merged_df,(1,1),copy_head = True,nan='')

In [ ]:
vehicle_assignment.sample(10)

,drn,plate_number,effective_start_date,effective_end_date,city,start_week,end_week
10988,MVS10741,CAA347334,2024-01-11,2024-02-21,Cape Town,2024-01-08,2024-02-25
10018,MVS14204,CAA349140,2024-01-12,2024-06-04,Cape Town,2024-01-08,2024-06-09
13724,MVS14352,CAA403295,2022-10-10,2023-05-18,Cape Town,2022-10-10,2023-05-21
10179,MVS17707,LK76KXGP,2024-01-29,2024-06-02,Johannesburg,2024-01-29,2024-06-02
1124,MVS21429,LL48XYGP,2025-04-25,2025-07-16,Johannesburg,2025-04-21,2025-07-20
10280,MVS17641,CAA403025,2024-01-26,2024-05-19,Cape Town,2024-01-22,2024-05-19
1385,MVS17874,KM70CFGP,2025-03-13,2025-07-16,Johannesburg,2025-03-10,2025-07-20
6074,MVS16542,KT30BRGP,2024-06-04,2025-02-05,Johannesburg,2024-06-03,2025-02-09
14781,MVS14827,KN00MPGP,2022-11-09,2022-12-07,Johannesburg,2022-11-07,2022-12-11
14603,MVS13465,CAA310899,2022-10-24,2023-01-13,Cape Town,2022-10-24,2023-01-15


In [ ]:

assignments_data = vehicle_assignment
# Converting date columns to datetime
assignments_data['effective_start_date'] = pd.to_datetime(assignments_data['effective_start_date'])
assignments_data['effective_end_date'] = pd.to_datetime(assignments_data['effective_end_date'])

# Filtering out all records with nulls in effective_end_date (currently active drivers)
active_drivers = assignments_data[assignments_data['effective_end_date'].isna()]

# Filtering to add drivers whose effective_end_date is at most 1 month old from the current date
one_month_ago = datetime.now() - timedelta(days=30)
recently_inactive_drivers = assignments_data[(assignments_data['effective_end_date'] >= one_month_ago) & (assignments_data['effective_end_date'].notna())]

# Combining active drivers and recently inactive drivers
combined_drivers = pd.concat([active_drivers, recently_inactive_drivers]).drop_duplicates(subset='drn')

# Ensuring we have a unique list of DRNs
unique_drns = combined_drivers.drop_duplicates(subset='drn')

# Dropping the 'plate_number', 'driver_name', and 'country' columns
final_drivers = unique_drns.drop(['start_week', 'end_week','city'], axis=1)
final_drivers.sample(10)

,drn,plate_number,effective_start_date,effective_end_date
611,MVS21755,CAA347427,2025-06-09,2025-07-16
2522,MVS17059,LK63HFGP,2024-02-06,2025-07-16
3286,MVS21399,CAA403177,2025-04-22,2025-06-20
1488,MVS20939,LW27LPGP,2025-02-25,2025-07-16
302,MVS18021,LL48XRGP,2025-06-30,2025-07-16
1860,MVS14212,KT53DMGP,2024-12-03,2025-07-16
247,MVS14830,KM58DSGP,2025-07-02,2025-07-16
458,MVS21917,LK55FMGP,2025-06-18,2025-07-16
167,MVS21902,LL48RFGP,2025-07-07,2025-07-16
2711,MVS21634,LW03LPGP,2025-05-20,2025-07-15


In [ ]:
active_loans_data = comp_sheet.worksheet_by_title("active_loans")
active_loans_data.set_dataframe(final_drivers,(1,1),copy_head = True,nan='')

In [ ]:
merged_df[merged_df['drn']=='MVS00102'].sort_values('week')

,drn,week,total_loan_repaid,plate_number,effective_start_date,effective_end_date,start_week,end_week,Adjusted Start Week,Adjusted End Week
60825,MVS00102,2023-05-15,1716.67,NaN,NaN,NaN,NaN,NaN,NaT,NaT
62697,MVS00102,2023-05-22,2575.00,NaN,NaN,NaN,NaN,NaN,NaT,NaT
63698,MVS00102,2023-05-29,429.16,KX76DGGP,2023-06-01,2023-11-06,2023-05-29,2023-11-12,2023-05-26,2023-11-15
64847,MVS00102,2023-06-05,2575.00,KX76DGGP,2023-06-01,2023-11-06,2023-05-29,2023-11-12,2023-05-26,2023-11-15
66152,MVS00102,2023-06-12,2575.00,KX76DGGP,2023-06-01,2023-11-06,2023-05-29,2023-11-12,2023-05-26,2023-11-15
...,...,...,...,...,...,...,...,...,...,...
306667,MVS00102,2025-06-02,2475.00,KX72BVGP,2025-04-15,2025-07-16,2025-04-14,2025-07-20,2025-04-11,2025-07-23
309694,MVS00102,2025-06-09,2475.00,KX72BVGP,2025-04-15,2025-07-16,2025-04-14,2025-07-20,2025-04-11,2025-07-23
315115,MVS00102,2025-06-16,2475.00,KX72BVGP,2025-04-15,2025-07-16,2025-04-14,2025-07-20,2025-04-11,2025-07-23
316541,MVS00102,2025-06-23,2475.00,KX72BVGP,2025-04-15,2025-07-16,2025-04-14,2025-07-20,2025-04-11,2025-07-23


In [ ]:
# Filtering rows where 'Plate_Number' is NaN
unmatched_records = merged_df[merged_df['plate_number'].isnull()]
unmatched_records.head(10)

,drn,week,total_loan_repaid,plate_number,effective_start_date,effective_end_date,start_week,end_week,Adjusted Start Week,Adjusted End Week
48,MVS10296,2021-09-20,2309.0,NaN,NaN,NaN,NaN,NaN,NaT,NaT
172,MVS10350,2021-09-27,1916.0,NaN,NaN,NaN,NaN,NaN,NaT,NaT
203,MVS10296,2021-09-27,2300.0,NaN,NaN,NaN,NaN,NaN,NaT,NaT
214,MVS10350,2021-10-04,2300.0,NaN,NaN,NaN,NaN,NaN,NaT,NaT
276,MVS10296,2021-10-04,2300.0,NaN,NaN,NaN,NaN,NaN,NaT,NaT
402,MVS10350,2021-10-11,2300.0,NaN,NaN,NaN,NaN,NaN,NaT,NaT
469,MVS10296,2021-10-11,2300.0,NaN,NaN,NaN,NaN,NaN,NaT,NaT
580,MVS10296,2021-10-18,2300.0,NaN,NaN,NaN,NaN,NaN,NaT,NaT
609,MVS10350,2021-10-18,2300.0,NaN,NaN,NaN,NaN,NaN,NaT,NaT
809,MVS10296,2021-10-25,2300.0,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [ ]:
unmatched_records.shape

(12186, 10)

In [ ]:
# Filtering records not in 'unmatched_records'
matched_records_df = merged_df[
    ~merged_df[['drn', 'week']].apply(tuple, axis=1).isin(unmatched_records[['drn', 'week']].apply(tuple, axis=1))
]

matched_records_df.sample(10)

,drn,week,total_loan_repaid,plate_number,effective_start_date,effective_end_date,start_week,end_week,Adjusted Start Week,Adjusted End Week
259870,MVS16628,2025-02-10,450.28,CAA403446,2024-06-27,2025-04-29,2024-06-24,2025-05-04,2024-06-21,2025-05-07
299073,MVS19942,2025-05-12,151.91,CAA402731,2024-11-13,2025-05-12,2024-11-11,2025-05-18,2024-11-08,2025-05-21
245972,MVS19834,2025-01-13,1287.50,KL09DDGP,2024-12-14,2025-07-16,2024-12-09,2025-07-20,2024-12-06,2025-07-23
300841,MVS17265,2025-05-19,2800.00,LK97ZMGP,2024-10-25,2025-07-16,2024-10-21,2025-07-20,2024-10-18,2025-07-23
57818,MVS12950,2023-04-24,2300.00,KL34XJGP,2022-05-25,2024-12-04,2022-05-23,2024-12-08,2022-05-20,2024-12-11
179353,MVS18032,2024-08-19,3100.00,LL12HLGP,2024-05-06,2025-07-15,2024-05-06,2025-07-20,2024-05-03,2025-07-23
40645,MVS11946,2023-01-02,203.75,KK85CHGP,2022-02-21,2023-12-26,2022-02-21,2023-12-31,2022-02-18,2024-01-03
124485,MVS14228,2024-02-26,2760.46,KT88MHGP,2022-09-27,2024-07-29,2022-09-26,2024-08-04,2022-09-23,2024-08-07
242513,MVS16831,2025-01-06,2669.24,JY76NFGP,2024-12-04,2025-05-06,2024-12-02,2025-05-11,2024-11-29,2025-05-14
205572,MVS16485,2024-10-14,2570.23,KX71ZKGP,2023-09-18,2024-11-09,2023-09-18,2024-11-10,2023-09-15,2024-11-13


In [ ]:
matched_records_df.shape

(309747, 10)

In [ ]:
matched_records_df[matched_records_df['drn'] == 'MVS00102'].sort_values('week')

,drn,week,total_loan_repaid,plate_number,effective_start_date,effective_end_date,start_week,end_week,Adjusted Start Week,Adjusted End Week
63698,MVS00102,2023-05-29,429.16,KX76DGGP,2023-06-01,2023-11-06,2023-05-29,2023-11-12,2023-05-26,2023-11-15
64847,MVS00102,2023-06-05,2575.00,KX76DGGP,2023-06-01,2023-11-06,2023-05-29,2023-11-12,2023-05-26,2023-11-15
66152,MVS00102,2023-06-12,2575.00,KX76DGGP,2023-06-01,2023-11-06,2023-05-29,2023-11-12,2023-05-26,2023-11-15
66956,MVS00102,2023-06-19,2575.00,KX76DGGP,2023-06-01,2023-11-06,2023-05-29,2023-11-12,2023-05-26,2023-11-15
68153,MVS00102,2023-06-26,2575.00,KX76DGGP,2023-06-01,2023-11-06,2023-05-29,2023-11-12,2023-05-26,2023-11-15
...,...,...,...,...,...,...,...,...,...,...
306667,MVS00102,2025-06-02,2475.00,KX72BVGP,2025-04-15,2025-07-16,2025-04-14,2025-07-20,2025-04-11,2025-07-23
309694,MVS00102,2025-06-09,2475.00,KX72BVGP,2025-04-15,2025-07-16,2025-04-14,2025-07-20,2025-04-11,2025-07-23
315115,MVS00102,2025-06-16,2475.00,KX72BVGP,2025-04-15,2025-07-16,2025-04-14,2025-07-20,2025-04-11,2025-07-23
316541,MVS00102,2025-06-23,2475.00,KX72BVGP,2025-04-15,2025-07-16,2025-04-14,2025-07-20,2025-04-11,2025-07-23


In [ ]:
# Group by Plate_Number, DRN, Pickup Date, and Drop-Off Date, and sum the total_remitted_for_the_loan_emi_opex_
matched_records_df = matched_records_df.groupby([ 'drn', 'plate_number','effective_start_date', 'effective_end_date'])['total_loan_repaid'].sum().reset_index()

matched_records_df[matched_records_df['drn'] == 'MVS00102'].sort_values('effective_start_date')

,drn,plate_number,effective_start_date,effective_end_date,total_loan_repaid
1,MVS00102,KX76DGGP,2023-06-01,2023-11-06,53256.650000
2,MVS00102,KY02FWGP,2023-12-21,2024-09-03,85793.310000
3,MVS00102,KY02FWGP,2024-09-05,2025-04-02,72519.983333
0,MVS00102,KX72BVGP,2025-04-15,2025-07-16,28462.500000


In [ ]:
# Writing to EMI Tenure Comparison Sheet tab
comp_sheet = client.open_by_key('1ThHVoubzA-WK-yFpXw7wA-_hWsxgcdwdZtuz7pSDX7A')
repaid_data = comp_sheet.worksheet_by_title("Loan Waterfall")
repaid_data.set_dataframe(matched_records_df,(1,1),copy_head = True,nan='')

In [ ]:
unmatched_records = unmatched_records[['drn','week', 'plate_number','effective_start_date', 'effective_end_date','total_loan_repaid']]
unmatched_records.head(10)

,drn,week,plate_number,effective_start_date,effective_end_date,total_loan_repaid
48,MVS10296,2021-09-20,NaN,NaN,NaN,2309.0
172,MVS10350,2021-09-27,NaN,NaN,NaN,1916.0
203,MVS10296,2021-09-27,NaN,NaN,NaN,2300.0
214,MVS10350,2021-10-04,NaN,NaN,NaN,2300.0
276,MVS10296,2021-10-04,NaN,NaN,NaN,2300.0
402,MVS10350,2021-10-11,NaN,NaN,NaN,2300.0
469,MVS10296,2021-10-11,NaN,NaN,NaN,2300.0
580,MVS10296,2021-10-18,NaN,NaN,NaN,2300.0
609,MVS10350,2021-10-18,NaN,NaN,NaN,2300.0
809,MVS10296,2021-10-25,NaN,NaN,NaN,2300.0


In [ ]:
# Writing to EMI Tenure Comparison Sheet tab
comp_sheet = client.open_by_key('1ThHVoubzA-WK-yFpXw7wA-_hWsxgcdwdZtuz7pSDX7A')
unmapped_repaid_data = comp_sheet.worksheet_by_title("unmapped")
unmapped_repaid_data.set_dataframe(unmatched_records,(1,1),copy_head = True,nan='')